In [ ]:
OPTIONS="--n_splits 1 --split 0 --nproc 2 --batch_size 20"
!python $HOME/MasterProject/Code/ClinicaTools/AD-DL/clinicaaddl/clinicaaddl/main.py train /u/horlavanasta/MasterProject/NNs_new/subject_model-SEResNet18_preprocessing-linear_task-AD_CN_norm-1_augmFalse_20210426_153458 --resume False --n_splits 1 --split 0 --nproc 1 --batch_size 35

In [22]:
def available_scans(source_dir, ext=".pt"):
    import os
    subject_ids=[]
    for (dirpath, dirnames, filenames) in os.walk(source_dir):
        for f in filenames:
            if f.endswith(ext):
                subject_ids.append(f)
    return subject_ids



In [26]:
scans=available_scans("../ADNI_data/OrigScans/", ext=".nii")
len(scans)

1075

In [27]:
import os
folders=[]
for f in os.scandir("../ADNI_data/OrigScans/ADNI/"):
    if 'sub-' in f.path:
        pass
    folders.append(f.path)
print(len(folders))

818


In [ ]:
!python ~/MasterProject/Code/ClinicaTools/AD-DL/clinicaaddl/clinicaaddl/main.py prepare_train image cnn ~/MasterProject/ADNI_data/CAPSPreprocessedT1linear t1-linear ~/MasterProject/ADNI_data/DataPrep/labels/train ~/MasterProject/NNs/model SEResNet18 --baseline

In [1]:
import os
folders=[]
for f in os.scandir("/u/horlavanasta/MasterProject/NNs_new"):
    if 'subject_model' in f.path:
        folders.append(f.path)
print(folders)

['/u/horlavanasta/MasterProject/NNs_new/subject_model-Conv5_FC3_preprocessing-linear_task-AD_CN_norm-1_augmTrue', '/u/horlavanasta/MasterProject/NNs_new/subject_model-ResNet18_preprocessing-linear_task-AD_CN_norm-1_augmTrue', '/u/horlavanasta/MasterProject/NNs_new/subject_model-ResNet18_preprocessing-linear_task-AD_CN_norm-1_augmFalse', '/u/horlavanasta/MasterProject/NNs_new/subject_model-SEResNet18_preprocessing-linear_task-AD_CN_norm-1_augmFalse', '/u/horlavanasta/MasterProject/NNs_new/subject_model-Conv5_FC3_preprocessing-linear_task-AD_CN_norm-1_augmFalse', '/u/horlavanasta/MasterProject/NNs_new/subject_model-ResNet18Expanded_preprocessing-linear_task-AD_CN_norm-1_augmFalse', '/u/horlavanasta/MasterProject/NNs_new/subject_model-ResNet18Expanded_preprocessing-linear_task-AD_CN_norm-1_augmTrue', '/u/horlavanasta/MasterProject/NNs_new/subject_model-SEResNet18_preprocessing-linear_task-AD_CN_norm-1_augmTrue_20210426_170806', '/u/horlavanasta/MasterProject/NNs_new/subject_model-SEResNet

In [ ]:
!python ~/MasterProject/Code/ClinicaTools/AD-DL/clinicaaddl/clinicaaddl/main.py train ~/MasterProject/NNs/model --n_splits 1 --split 0                        

In [2]:
for f in folders:
    !echo $f
    !python ~/MasterProject/Code/ClinicaTools/AD-DL/clinicaaddl/clinicaaddl/main.py classify ~/MasterProject/ADNI_data/CAPSPreprocessedT1linear ~/MasterProject/ADNI_data/DataPrep/labels_new/test $f test

/u/horlavanasta/MasterProject/NNs_new/subject_model-Conv5_FC3_preprocessing-linear_task-AD_CN_norm-1_augmTrue
/u/horlavanasta/MasterProject/NNs_new/subject_model-ResNet18_preprocessing-linear_task-AD_CN_norm-1_augmTrue
/u/horlavanasta/MasterProject/NNs_new/subject_model-ResNet18_preprocessing-linear_task-AD_CN_norm-1_augmFalse
/u/horlavanasta/MasterProject/NNs_new/subject_model-SEResNet18_preprocessing-linear_task-AD_CN_norm-1_augmFalse
/u/horlavanasta/MasterProject/NNs_new/subject_model-Conv5_FC3_preprocessing-linear_task-AD_CN_norm-1_augmFalse
/u/horlavanasta/MasterProject/NNs_new/subject_model-ResNet18Expanded_preprocessing-linear_task-AD_CN_norm-1_augmFalse
/u/horlavanasta/MasterProject/NNs_new/subject_model-ResNet18Expanded_preprocessing-linear_task-AD_CN_norm-1_augmTrue
/u/horlavanasta/MasterProject/NNs_new/subject_model-SEResNet18_preprocessing-linear_task-AD_CN_norm-1_augmTrue_20210426_170806
/u/horlavanasta/MasterProject/NNs_new/subject_model-SEResNet18_preprocessing-linear_ta

In [34]:
log_path

'/u/horlavanasta/MasterProject/NNs/subject_model-Conv5_FC3_preprocessing-linear_task-AD_CN_norm-1_augm1-t-0/fold-0/tensorboard_logs/train/events.out.tfevents.1619017713.co6059'

In [2]:
%load_ext tensorboard

In [49]:
readable_params=['model', 'data_augmentation', 'batch_size', 'learning_rate', "loss"]
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import json
import numpy as np
from matplotlib import cm

def plot_loss_with_results(params, results, history, saved_file_path=None):
    
    fig, axes = plt.subplots(2, 2, figsize=(20, 18))
    str_suptitle="Params: "
    
    for i, line in enumerate(readable_params):
        str_suptitle+=line+': '+str(params[line])+"; "
        
    for k,mode in enumerate(results.keys()):
        plot_bar_plots(axes[0][k], results[mode], mode)
        
    plot_history(axes[1][0], history, mode='loss')
    plot_history(axes[1][1], history, mode='balanced_accuracy')


    
    plt.suptitle(str_suptitle)
#     plt.tight_layout()

#     plt.show()
    if saved_file_path is not None:
        plt.savefig(saved_file_path)
    else:
        plt.show()
        
    plt.close()
        
def plot_history(ax, history, mode):
    def find_best(arr, ismin=True):
        arr=np.array(arr)
        if ismin:
            best_loss_idx_train=np.where(arr == np.amin(arr))[0][0]
        else:
            best_loss_idx_train=np.where(arr == np.amax(arr))[0][0]
        return best_loss_idx_train, arr[best_loss_idx_train]
            
    ax.plot(history["epoch"], history[mode+"_train"], 'black', lw=1, label='train '+mode)
    ax.plot(history["epoch"], history[mode+"_valid"], 'red', lw=1, label='validation '+mode)
    
    idx, val=find_best(history[mode+"_valid"], mode=='loss')
    ax.plot(idx, val,'o', color='black')
    
#     ax.legend()
    ax.legend(bbox_to_anchor=(0.5, -0.15), loc='lower center',
         ncol=2, fontsize ='large')
    
    if mode=='loss' and np.max(history[mode+"_valid"]) > 0.5:
        ax.set_ylim(bottom=-0.001, top=0.5)
    if mode=='balanced_accuracy':
        ax.set_ylim(bottom=-0.001, top=1.1)
    ax.set_title(mode)
    
def plot_bar_plots(ax, results, mode):
    
    def autolabel(ax, rects):
        for rect in rects:

            height = rect.get_height()
            ax.text(rect.get_x() + rect.get_width()/2., 1.02*height,
                    '%.3f' % (height),
                    ha='center', va='bottom')
        
    def reshape_results(results):
        metrics={}
        for key in results["train"].keys():
            metrics[key]=[]
        for mode in results.keys():
            for metric in results[mode].keys():
                metrics[metric].append(results[mode][metric][0])
        return metrics
    
    ax.set_ylim(bottom=-0.001, top=1.1)
    results_transposed = reshape_results(results)
    N=len(results.keys())
    
    ind = np.arange(N)
    width = 0.2       
    width_ratio=0.9
#     ax.set_prop_cycle('color', Pastel1_4.mpl_colors)
    ax.set_prop_cycle('color', cm.get_cmap('Paired').colors)

    
    for i, key in enumerate(results_transposed.keys()):
        autolabel(ax, ax.bar(ind+width*i, results_transposed[key] , width*width_ratio, label=key.capitalize(), edgecolor='dimgrey'))
    
    xstips_position=ind + width

    xsticklabels=[disease_type for disease_type in results.keys()]
    ax.set_xticklabels(xsticklabels)
    ax.set_xticks(xstips_position)
    ax.legend(bbox_to_anchor=(0.5, -0.15), loc='lower center',
         ncol=3, fontsize ='large')
    ax.set_title("Model from: "+ mode)

In [25]:
progress_list=["/u/horlavanasta/MasterProject/NNs_new/subject_model-ResNet18Expanded_preprocessing-linear_task-AD_CN_norm-1_loss-default_augmTrue", 
               "/u/horlavanasta/MasterProject/NNs_new/subject_model-Conv5_FC3_preprocessing-linear_task-AD_CN_norm-1_loss-default_augmTrue",
"/u/horlavanasta/MasterProject/NNs_new/subject_model-ResNet18_preprocessing-linear_task-AD_CN_norm-1_loss-default_augmTrue",
"/u/horlavanasta/MasterProject/NNs/subject_model-SEResNet18_preprocessing-linear_task-AD_CN_norm-1_loss-WeightedCrossEntropy_augmTrue",
"/u/horlavanasta/MasterProject/NNs_new/subject_model-SEResNet18_preprocessing-linear_task-AD_CN_norm-1_loss-WeightedCrossEntropy_augmTrue",
"/u/horlavanasta/MasterProject/NNs_new/subject_model-Conv5_FC3_preprocessing-linear_task-AD_CN_norm-1_augmTrue"]

In [26]:
progress_list=["/u/horlavanasta/MasterProject/NNs_new/subject_model-Conv5_FC3_preprocessing-linear_task-AD_CN_norm-1_augmTrue"]

In [28]:
f="/u/horlavanasta/MasterProject/NNs_new/subject_model-Conv5_FC3_preprocessing-linear_task-AD_CN_norm-1_augmTrue"
!python ~/MasterProject/Code/ClinicaTools/AD-DL/clinicaaddl/clinicaaddl/main.py classify ~/MasterProject/ADNI_data/CAPSPreprocessedT1linear ~/MasterProject/ADNI_data/DataPrep/labels_new/test $f test

In [27]:
import pathlib
import pandas as pd
import os

folders=[]
for f in os.scandir("/u/horlavanasta/MasterProject/NNs_new"):
    if 'subject_model' in f.path and f.path not in progress_list:
        
        folders.append(f.path)
        
for f in folders:
    !echo $f
    !python ~/MasterProject/Code/ClinicaTools/AD-DL/clinicaaddl/clinicaaddl/main.py classify ~/MasterProject/ADNI_data/CAPSPreprocessedT1linear ~/MasterProject/ADNI_data/DataPrep/labels_new/test $f test
 
folders2=[]
for f in os.scandir("/u/horlavanasta/MasterProject/NNs"):
    if 'subject_model' in f.path and f.path not in progress_list:
        folders2.append(f.path)
        
for f in folders2:
    !echo $f
    !python ~/MasterProject/Code/ClinicaTools/AD-DL/clinicaaddl/clinicaaddl/main.py classify ~/MasterProject/ADNI_data/CAPSPreprocessedT1linear ~/MasterProject/ADNI_data/DataPrep/labels/test $f test

/u/horlavanasta/MasterProject/NNs_new/subject_model-SEResNet18_preprocessing-linear_task-AD_CN_norm-1_loss-default_augmFalse
/u/horlavanasta/MasterProject/NNs_new/subject_model-ResNet18_preprocessing-linear_task-AD_CN_norm-1_augmTrue
/u/horlavanasta/MasterProject/NNs_new/subject_model-ResNet18_preprocessing-linear_task-AD_CN_norm-1_augmFalse
/u/horlavanasta/MasterProject/NNs_new/subject_model-SEResNet18Expanded_preprocessing-linear_task-AD_CN_norm-1_augmTrue
/u/horlavanasta/MasterProject/NNs_new/subject_model-SEResNet18_preprocessing-linear_task-AD_CN_norm-1_augmFalse
/u/horlavanasta/MasterProject/NNs_new/subject_model-ResNet18Expanded_preprocessing-linear_task-AD_CN_norm-1_loss-default_augmTrue
/u/horlavanasta/MasterProject/NNs_new/subject_model-SEResNet18Expanded_preprocessing-linear_task-AD_CN_norm-1_augmFalse
/u/horlavanasta/MasterProject/NNs_new/subject_model-Conv5_FC3_preprocessing-linear_task-AD_CN_norm-1_augmFalse
/u/horlavanasta/MasterProject/NNs_new/subject_model-ResNet18Expa

In [16]:
len(folders)

17

In [17]:
folders=[]
for f in os.scandir("/u/horlavanasta/MasterProject/NNs_new"):
    if 'subject_model' in f.path:
        folders.append(f.path)

len(folders)

22

In [30]:
progress_list=["/u/horlavanasta/MasterProject/NNs_new/subject_model-SEResNet18_preprocessing-linear_task-AD_CN_norm-1_loss-default_augmTrue"]

In [68]:
import pathlib
import pandas as pd
import os

folders=[]
for f in os.scandir("/u/horlavanasta/MasterProject/NNs"):
    if 'subject_model' in f.path and f.path not in progress_list:
        folders.append(f.path)
  
        
selection_metrics=["best_loss", "best_balanced_accuracy"]
modes=['train', 'validation', 'test']
model_types=["SEResNet18", "ResNet18","Conv5_FC3", "SEResNet18Expanded","ResNet18Expanded"]
reported_best_accuracies={}
for model_type in model_types:
    reported_best_accuracies[model_type]={"max_value":0,"mode":"", "results":{}, "params":{}, "history":{},"model_name":{} }

results_folder='/u/horlavanasta/MasterProject/Code/ClinicaTools/AD-DL/results/test-0.25/'
for f in folders[:]:
    currentDirectory = pathlib.Path(f)
#     print("currentDirectory")

    # Search for 'fold-*' pattern
    currentPattern = "fold-*"
    path_params=os.path.join(currentDirectory,"commandline_train.json")
    
    with open(path_params, "r") as f:
            params = json.load(f)
            
    # loop depending the number of folds found in the model folder
    for fold_dir in currentDirectory.glob(currentPattern):
        fold = int(str(fold_dir).split("-")[-1])
        results={}
        for selection_metric in selection_metrics:
            results[selection_metric]={}
            history = pd.read_csv(os.path.join(currentDirectory, 'fold-%i' % fold, 'training.tsv'), sep='\t')
            for mode in modes:
                performance_dir = os.path.join(currentDirectory, 'fold-%i' % fold, 'cnn_classification')
                test_diagnosis_path=os.path.join(performance_dir, selection_metric,'%s_image_level_metrics.tsv' % (mode))
                test_diagnosis_df = pd.read_csv(test_diagnosis_path, sep='\t')
                results[selection_metric][mode]=test_diagnosis_df[["sensitivity", 'specificity', 'balanced_accuracy', 'accuracy']]
        
        folder_type='barplots_with_loss'
        preprocessing=params["preprocessing"]

        augmentations_presense='augmentations' if params['data_augmentation'] else "no_augmentations"
        network_type=params['model']
        if "loss" not in params.keys():
            params['loss']='default'

        loss=params['loss']
        
        name=os.path.basename(os.path.normpath(currentDirectory))

        path=os.path.join(results_folder, folder_type, preprocessing, network_type, loss, augmentations_presense)
        os.makedirs(path, exist_ok=True)
        file_name='barplots_'+name+'.png'
        
        plot_loss_with_results(params, results, history, os.path.join(path, file_name))
        for mode in results.keys():
            if results[mode]["test"]["balanced_accuracy"][0]>reported_best_accuracies[network_type]["max_value"]:
                reported_best_accuracies[network_type]["max_value"]=results[mode]["test"]["balanced_accuracy"][0]
                reported_best_accuracies[network_type]["mode"]=mode
                reported_best_accuracies[network_type]["results"]=results
                reported_best_accuracies[network_type]["params"]=params
                reported_best_accuracies[network_type]["history"]=history
                reported_best_accuracies[network_type]["model_name"]=name 
                
            
best_results_path=os.path.join(results_folder, "best_results")

os.makedirs(best_results_path, exist_ok=True)
for network_type in model_types:
    
#     print(reported_best_accuracies[network_type]["params"])
    if reported_best_accuracies[network_type]["max_value"]>0.0:
        print(network_type, reported_best_accuracies[network_type]["params"]["loss"],  reported_best_accuracies[network_type]["mode"], reported_best_accuracies[network_type]["max_value"])

        filepath=os.path.join(best_results_path, reported_best_accuracies[network_type]["model_name"]) 
        plot_loss_with_results(reported_best_accuracies[network_type]["params"], reported_best_accuracies[network_type]["results"], reported_best_accuracies[network_type]["history"], filepath)


        
        
#     print("__________________________________________________________________________________")
            
    

SEResNet18 WeightedCrossEntropy best_balanced_accuracy 0.8090705487122061
ResNet18 WeightedCrossEntropy best_balanced_accuracy 0.8197088465845465
Conv5_FC3 WeightedCrossEntropy best_loss 0.8409854423292273


In [67]:
import pathlib
import pandas as pd
import os

folders=[]
for f in os.scandir("/u/horlavanasta/MasterProject/NNs_new"):
    if 'subject_model' in f.path and f.path not in progress_list:
        folders.append(f.path)
  
        
selection_metrics=["best_loss", "best_balanced_accuracy"]
modes=['train', 'validation', 'test']
model_types=["SEResNet18", "ResNet18","Conv5_FC3", "SEResNet18Expanded","ResNet18Expanded",]
reported_best_accuracies={}
for model_type in model_types:
    reported_best_accuracies[model_type]={"max_value":0,"mode":"", "results":{}, "params":{}, "history":{},"model_name":"" }

results_folder='/u/horlavanasta/MasterProject/Code/ClinicaTools/AD-DL/results/test-0.25/'
for f in folders[:]:
    currentDirectory = pathlib.Path(f)
#     print("currentDirectory")

    # Search for 'fold-*' pattern
    currentPattern = "fold-*"
    path_params=os.path.join(currentDirectory,"commandline_train.json")
    
    with open(path_params, "r") as f:
            params = json.load(f)
            
    # loop depending the number of folds found in the model folder
    for fold_dir in currentDirectory.glob(currentPattern):
        fold = int(str(fold_dir).split("-")[-1])
        results={}
        for selection_metric in selection_metrics:
            results[selection_metric]={}
            history = pd.read_csv(os.path.join(currentDirectory, 'fold-%i' % fold, 'training.tsv'), sep='\t')
            for mode in modes:
                performance_dir = os.path.join(currentDirectory, 'fold-%i' % fold, 'cnn_classification')
                test_diagnosis_path=os.path.join(performance_dir, selection_metric,'%s_image_level_metrics.tsv' % (mode))
                test_diagnosis_df = pd.read_csv(test_diagnosis_path, sep='\t')
                results[selection_metric][mode]=test_diagnosis_df[["sensitivity", 'specificity', 'balanced_accuracy', 'accuracy']]
        
        folder_type='barplots_with_loss'
        preprocessing=params["preprocessing"]

        augmentations_presense='augmentations' if params['data_augmentation'] else "no_augmentations"
        network_type=params['model']
        if "loss" not in params.keys():
            params['loss']='default'

        loss=params['loss']
        
        name=os.path.basename(os.path.normpath(currentDirectory))

        path=os.path.join(results_folder, folder_type, preprocessing, network_type, loss, augmentations_presense)
        os.makedirs(path, exist_ok=True)
        file_name='barplots_'+name+'.png'
        
        plot_loss_with_results(params, results, history, os.path.join(path, file_name))
        for mode in results.keys():
            if results[mode]["test"]["balanced_accuracy"][0]>reported_best_accuracies[network_type]["max_value"]:
                reported_best_accuracies[network_type]["max_value"]=results[mode]["test"]["balanced_accuracy"][0]
                reported_best_accuracies[network_type]["mode"]=mode
                reported_best_accuracies[network_type]["results"]=results
                reported_best_accuracies[network_type]["params"]=params
                reported_best_accuracies[network_type]["history"]=history
                reported_best_accuracies[network_type]["model_name"]=name 
                
best_results_path=os.path.join(results_folder, "best_results")

os.makedirs(best_results_path, exist_ok=True)
for network_type in model_types:
    print(network_type, reported_best_accuracies[network_type]["params"]["loss"], reported_best_accuracies[network_type]["mode"], reported_best_accuracies[network_type]["max_value"])
    
#     print(reported_best_accuracies[network_type]["params"])
    if reported_best_accuracies[network_type]["max_value"]>0.0:
        filepath=os.path.join(best_results_path, reported_best_accuracies[network_type]["model_name"]) 
        plot_loss_with_results(reported_best_accuracies[network_type]["params"], reported_best_accuracies[network_type]["results"], reported_best_accuracies[network_type]["history"], filepath)

        
        
#     print("__________________________________________________________________________________")
            
    

SEResNet18 WeightedCrossEntropy best_balanced_accuracy 0.9246411483253588
ResNet18 default best_balanced_accuracy 0.9473684210526316
Conv5_FC3 WeightedCrossEntropy best_balanced_accuracy 0.9246411483253588
SEResNet18Expanded WeightedCrossEntropy best_loss 0.9246411483253588
ResNet18Expanded WeightedCrossEntropy best_loss 0.931818181818182


In [4]:
HIDA_participant=True
name="Nastassya"

In [9]:
thingsInCommon=["Based in Germany", "Master Studies", "Pytorch"]
if HIDA_participant and name in ["Konstantin", "Nastassya", "Moiz", "Tobias"]:
    print("Stuff in common:")
    for i, thingInCommon in enumerate(thingsInCommon):
        print(i, thingInCommon)
    

Stuff in common:
0 Based in Germany
1 Master Studies
2 Pytorch
